In [1]:
import numpy as np
import pandas as pd
import geopandas as gp
import matplotlib.pyplot as plt

<frozen importlib._bootstrap>:219: RuntimeWarning: numpy.ufunc size changed, may indicate binary incompatibility. Expected 192 from C header, got 216 from PyObject


In [53]:
nl_covid = pd.read_csv('../data/Raw/corona/nl_corona.csv',sep='\t')
weather = pd.read_csv('../data/Raw/weather/weather.csv',sep='\t')
nl_weather = weather['iso3166-2'].str[:2] == 'NL'
nl_weather_df = weather[nl_weather]
nl_weather_df

,date,iso3166-2,RelativeHumiditySurface,SolarRadiation,Surfacepressure,TemperatureAboveGround,Totalprecipitation,UVIndex,WindSpeed
21,2020-02-13,NL-DR,91.711788,1.066942e+05,2.403207e+06,276.928340,0.010772,0.261780,4.431883
22,2020-02-13,NL-FL,89.725649,6.051013e+05,2.402642e+06,278.164065,0.012001,0.785507,6.057862
23,2020-02-13,NL-FR,91.979761,3.386435e+05,2.402547e+06,277.471187,0.009652,0.898502,5.811592
24,2020-02-13,NL-GE,88.672120,5.364181e+05,2.400907e+06,278.275055,0.012576,2.116279,4.602913
25,2020-02-13,NL-GR,91.948050,4.413943e+04,2.404533e+06,276.695832,0.009448,0.850829,4.974551
...,...,...,...,...,...,...,...,...,...
14878,2020-11-14,NL-NH,90.139789,3.315327e+05,2.428772e+06,284.412821,0.002788,0.022989,5.937843
14879,2020-11-14,NL-OV,90.624991,8.355392e+05,2.428846e+06,284.984318,0.001212,0.434870,3.534711
14880,2020-11-14,NL-UT,89.424252,9.894519e+05,2.429694e+06,285.008331,0.002306,0.135371,4.576365
14881,2020-11-14,NL-ZE,87.590779,1.583508e+06,2.429062e+06,285.820956,0.002250,2.522796,6.111993


In [56]:
print(nl_weather_df['iso3166-2'].unique())

['NL-DR' 'NL-FL' 'NL-FR' 'NL-GE' 'NL-GR' 'NL-LI' 'NL-NB' 'NL-NH' 'NL-OV'
 'NL-UT' 'NL-ZE' 'NL-ZH']
